In [ ]:
This resources were helpful:
* https://www.kaggle.com/code/ingbiodanielh/time-series-classification/notebook
* https://www.kaggle.com/code/beagle01/prediction-with-gradient-boosting-classifier/notebook 

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [2]:
avocado = pd.read_csv('avocado.csv')

In [3]:
avocado

,Unnamed: 0,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18244,7,2018-02-04,1.63,17074.83,2046.96,1529.20,0.00,13498.67,13066.82,431.85,0.0,organic,2018,WestTexNewMexico
18245,8,2018-01-28,1.71,13888.04,1191.70,3431.50,0.00,9264.84,8940.04,324.80,0.0,organic,2018,WestTexNewMexico
18246,9,2018-01-21,1.87,13766.76,1191.92,2452.79,727.94,9394.11,9351.80,42.31,0.0,organic,2018,WestTexNewMexico
18247,10,2018-01-14,1.93,16205.22,1527.63,2981.04,727.01,10969.54,10919.54,50.00,0.0,organic,2018,WestTexNewMexico


In [4]:
avocado.dtypes

Unnamed: 0        int64
Date             object
AveragePrice    float64
Total Volume    float64
4046            float64
4225            float64
4770            float64
Total Bags      float64
Small Bags      float64
Large Bags      float64
XLarge Bags     float64
type             object
year              int64
region           object
dtype: object

In [5]:
avocado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18249 entries, 0 to 18248
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    18249 non-null  int64  
 1   Date          18249 non-null  object 
 2   AveragePrice  18249 non-null  float64
 3   Total Volume  18249 non-null  float64
 4   4046          18249 non-null  float64
 5   4225          18249 non-null  float64
 6   4770          18249 non-null  float64
 7   Total Bags    18249 non-null  float64
 8   Small Bags    18249 non-null  float64
 9   Large Bags    18249 non-null  float64
 10  XLarge Bags   18249 non-null  float64
 11  type          18249 non-null  object 
 12  year          18249 non-null  int64  
 13  region        18249 non-null  object 
dtypes: float64(9), int64(2), object(3)
memory usage: 1.9+ MB


target = ['AveragePrice']
features = [feature for feature in avocado.columns if feature not in target]
avocado.head()

In [6]:
avocado.drop(['Unnamed: 0'], axis=1, inplace=True)

In [7]:
#avocado = pd.get_dummies(avocado,columns=['type', 'region'])
avocado = pd.get_dummies(avocado,columns=['type'])

In [8]:
avocado['Date'] = pd.to_datetime(avocado.Date)
avocado = avocado.set_index(avocado.Date)
avocado.drop('Date', axis = 1, inplace = True)
print('Column datatypes= \n',avocado.dtypes)
avocado

Column datatypes= 
 AveragePrice         float64
Total Volume         float64
4046                 float64
4225                 float64
4770                 float64
Total Bags           float64
Small Bags           float64
Large Bags           float64
XLarge Bags          float64
year                   int64
region                object
type_conventional      uint8
type_organic           uint8
dtype: object


,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,year,region,type_conventional,type_organic
Date,,,,,,,,,,,,,
2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,2015,Albany,1,0
2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,2015,Albany,1,0
2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,2015,Albany,1,0
2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,2015,Albany,1,0
2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,2015,Albany,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-02-04,1.63,17074.83,2046.96,1529.20,0.00,13498.67,13066.82,431.85,0.0,2018,WestTexNewMexico,0,1
2018-01-28,1.71,13888.04,1191.70,3431.50,0.00,9264.84,8940.04,324.80,0.0,2018,WestTexNewMexico,0,1
2018-01-21,1.87,13766.76,1191.92,2452.79,727.94,9394.11,9351.80,42.31,0.0,2018,WestTexNewMexico,0,1


In [9]:
# country
print(avocado['region'].value_counts())
#plt.figure(figsize=(22,10))
#sns.countplot(avocado['region'])
#plt.show()

Albany                 338
Sacramento             338
Northeast              338
NorthernNewEngland     338
Orlando                338
Philadelphia           338
PhoenixTucson          338
Pittsburgh             338
Plains                 338
Portland               338
RaleighGreensboro      338
RichmondNorfolk        338
Roanoke                338
SanDiego               338
Atlanta                338
SanFrancisco           338
Seattle                338
SouthCarolina          338
SouthCentral           338
Southeast              338
Spokane                338
StLouis                338
Syracuse               338
Tampa                  338
TotalUS                338
West                   338
NewYork                338
NewOrleansMobile       338
Nashville              338
Midsouth               338
BaltimoreWashington    338
Boise                  338
Boston                 338
BuffaloRochester       338
California             338
Charlotte              338
Chicago                338
C

In [10]:
X= avocado.iloc[:,:-3]
X.head()

,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,year
Date,,,,,,,,,,
2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,2015
2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,2015
2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,2015
2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,2015
2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,2015


In [11]:
y = avocado.iloc[:,-3]
print(f" Classes: {set(y)} ")
y.value_counts()

 Classes: {'BaltimoreWashington', 'Atlanta', 'RaleighGreensboro', 'Orlando', 'Nashville', 'Philadelphia', 'SanFrancisco', 'Tampa', 'Northeast', 'Indianapolis', 'RichmondNorfolk', 'StLouis', 'TotalUS', 'Spokane', 'PhoenixTucson', 'Midsouth', 'Detroit', 'Southeast', 'SanDiego', 'Jacksonville', 'NewOrleansMobile', 'MiamiFtLauderdale', 'Portland', 'DallasFtWorth', 'Columbus', 'NorthernNewEngland', 'BuffaloRochester', 'Seattle', 'Syracuse', 'Boston', 'GreatLakes', 'Boise', 'Louisville', 'West', 'NewYork', 'Chicago', 'HartfordSpringfield', 'LasVegas', 'Plains', 'LosAngeles', 'GrandRapids', 'Denver', 'SouthCentral', 'WestTexNewMexico', 'Albany', 'HarrisburgScranton', 'CincinnatiDayton', 'Pittsburgh', 'California', 'Charlotte', 'Roanoke', 'Sacramento', 'Houston', 'SouthCarolina'} 


Albany                 338
Sacramento             338
Northeast              338
NorthernNewEngland     338
Orlando                338
Philadelphia           338
PhoenixTucson          338
Pittsburgh             338
Plains                 338
Portland               338
RaleighGreensboro      338
RichmondNorfolk        338
Roanoke                338
SanDiego               338
Atlanta                338
SanFrancisco           338
Seattle                338
SouthCarolina          338
SouthCentral           338
Southeast              338
Spokane                338
StLouis                338
Syracuse               338
Tampa                  338
TotalUS                338
West                   338
NewYork                338
NewOrleansMobile       338
Nashville              338
Midsouth               338
BaltimoreWashington    338
Boise                  338
Boston                 338
BuffaloRochester       338
California             338
Charlotte              338
Chicago                338
C

In [12]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

In [13]:
y_train= y_train.values.reshape(-1,1)
y_test= y_test.values.reshape(-1,1)

from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.fit_transform(X_test)

In [28]:
X_train.shape, X_test.shape

((10949, 10), (7300, 10))

In [29]:
y_train.shape, y_test.shape 

((10949, 1), (7300, 1))

In [31]:
# import machine learning algorithms
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

In [32]:
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    gb = GradientBoostingClassifier(n_estimators=20, learning_rate = learning_rate, max_features=2, max_depth = 2, random_state = 0)
    gb.fit(X_train, y_train)
    print("Learning rate: ", learning_rate)
    print("Accuracy score (training): {0:.3f}".format(gb.score(X_train, y_train)))
    print("Accuracy score (validation): {0:.3f}".format(gb.score(X_test, y_test)))
    print()

C:\Users\Usuario\anaconda3\envs\spyder1\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Learning rate:  0.05
Accuracy score (training): 0.575
Accuracy score (validation): 0.318



C:\Users\Usuario\anaconda3\envs\spyder1\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Learning rate:  0.1
Accuracy score (training): 0.676
Accuracy score (validation): 0.347



C:\Users\Usuario\anaconda3\envs\spyder1\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

Learning rate:  0.5
Accuracy score (training): 0.052
Accuracy score (validation): 0.037



C:\Users\Usuario\anaconda3\envs\spyder1\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Learning rate:  0.75
Accuracy score (training): 0.036
Accuracy score (validation): 0.034



C:\Users\Usuario\anaconda3\envs\spyder1\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Learning rate:  1
Accuracy score (training): 0.031
Accuracy score (validation): 0.023



In [33]:
# Output confusion matrix and classification report of Gradient Boosting algorithm on validation set

gb = GradientBoostingClassifier(n_estimators=20, learning_rate = 0.25, max_features=2, max_depth = 2, random_state = 0)
gb.fit(X_train, y_train)
predictions = gb.predict(X_test)

print("Confusion Matrix:")
print(confusion_matrix(y_test, predictions))
print()
print("Classification Report")
print(classification_report(y_test, predictions))

C:\Users\Usuario\anaconda3\envs\spyder1\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Confusion Matrix:
[[  4   0   0 ...   0   0   0]
 [  0  70   2 ...   0   2   3]
 [  0   0  64 ...   0   0   0]
 ...
 [  0   0   0 ... 114   0   0]
 [  0   0   0 ...   0  99   0]
 [  0   3   8 ...   0   0  46]]

Classification Report
                     precision    recall  f1-score   support

             Albany       0.57      0.03      0.06       120
            Atlanta       0.44      0.49      0.47       143
BaltimoreWashington       0.32      0.48      0.39       133
              Boise       0.69      0.26      0.38       142
             Boston       0.76      0.27      0.40       144
   BuffaloRochester       0.39      0.32      0.35       125
         California       0.86      0.50      0.63       114
          Charlotte       0.74      0.37      0.49       149
            Chicago       0.92      0.47      0.62       137
   CincinnatiDayton       0.78      0.36      0.49       130
           Columbus       0.72      0.25      0.37       137
      DallasFtWorth       0.59    

In [18]:
model_rfc= RandomForestClassifier(n_estimators=100)
model_gbc = GradientBoostingClassifier() 

In [19]:
model_rfc.fit(X_train, y_train)

C:\Users\Usuario\AppData\Local\Temp/ipykernel_2612/2102309512.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_rfc.fit(X_train, y_train)


RandomForestClassifier()

In [20]:
model_gbc.fit(X_train, y_train)

C:\Users\Usuario\anaconda3\envs\spyder1\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingClassifier()

-------------------------------------------------------------------------------------------------------------------------------

### Evaluation

In [22]:
predict_region_rfc= model_rfc.predict(X_test)

In [23]:
predict_region_gbc= model_gbc.predict(X_test)

In [26]:
print(classification_report(y_test, predict_region_gbc))

                     precision    recall  f1-score   support

             Albany       0.35      0.20      0.26       120
            Atlanta       0.54      0.46      0.50       143
BaltimoreWashington       0.11      0.64      0.19       133
              Boise       0.96      0.32      0.48       142
             Boston       0.90      0.38      0.54       144
   BuffaloRochester       0.88      0.39      0.54       125
         California       0.91      0.52      0.66       114
          Charlotte       0.79      0.35      0.48       149
            Chicago       0.97      0.44      0.60       137
   CincinnatiDayton       0.91      0.41      0.56       130
           Columbus       0.85      0.29      0.43       137
      DallasFtWorth       0.77      0.36      0.49       135
             Denver       0.19      0.49      0.27       144
            Detroit       0.77      0.37      0.50       139
        GrandRapids       0.14      0.01      0.01       127
         GreatLakes    

In [27]:
print(classification_report(y_test, predict_region_rfc))

NameError: name 'predict_region_rfc' is not defined